📊 **Analysis:** Delivery Route Efficiency & Partner Earning Optimization

**Objective:** To evaluate the performance of delivery partners across different route tiers and ensure data integrity in financial reporting by handling missing earning metrics.

**Business Logic:**

- **Mean Imputation:** Employs subqueries to perform data imputation, replacing NULL earning values with the global average to maintain statistical accuracy.

- **Efficiency Tiering**: Categorizes delivery routes into performance segments based on the duration of delivery, enabling targeted operational improvements.

- **Pickup Density Analysis**: Calculates the "Earnings per Pickup" ratio to identify the most lucrative route patterns for logistics planning.

- **SubQuery**: Refines raw datasets by filtering for relevant attributes, reducing data complexity for downstream analysis.

- **CTEs**: Enhances code modularity and performance by utilizing Common Table Expressions (CTEs) for multi-stage processing.

- **SUM Function**: To sum of data column

- **CASE WHEN**: Implements conditional logic (CASE WHEN) to dynamically categorize routes based on performance thresholds.

Company:
Uber
◆
Difficulty:
Medium
◆
[Question Link](https://www.interviewmaster.ai/question/eats-delivery-partner-route-performance)

You are a Product Analyst investigating how delivery partners manage multiple order pickups. The team wants to understand the efficiency of order clustering and routing strategies. The goal is to optimize delivery route performance to support partner earnings and operational effectiveness.


**Tables**

**fct_delivery_routes**(route_id, delivery_partner_id, pickup_count, delivery_time, earnings, route_date)

**Question 1 of 3**

For all delivery routes between October 1st and December 31st, 2024, what percentage of routes had multiple (ie. 2 or more) order pickups? This metric will quantify how often order bundling occurs to help evaluate routing efficiency.

In [ ]:
# Finding total what percentage routes had multiple order pickups in Q4/2024 for evaluate routing efficiency.

SELECT (COUNT(route_id)::NUMERIC/(
    SELECT COUNT(*)
    FROM fct_delivery_routes
    WHERE route_date BETWEEN
      '2024-10-01' AND '2024-12-31')::NUMERIC*100) AS percentage_multiple_pickups
FROM fct_delivery_routes
WHERE (route_date BETWEEN '2024-10-01' AND '2024-12-31') AND
    (pickup_count >= 2);

**Question 2 of 3**

For delivery routes with multiple pickups between October 1st and December 31st, 2024, how does the average delivery time differ between routes with exactly 2 orders and routes with 3 or more orders? Use a CASE statement to segment the routes accordingly. This analysis will clarify the impact of different levels of order clustering on delivery performance.

In [ ]:
# Finding time difference between 2 orders per routes and 3 or more orders per routes for find impact.

WITH delivery_differ AS (
  SELECT *,
    CASE
      WHEN pickup_count = 2 THEN 'Two Pickups'
      WHEN pickup_count >= 3 THEN 'Three or More Pickups'
    END AS category
  FROM fct_delivery_routes
  WHERE (route_date BETWEEN '2024-10-01' AND '2024-12-31') AND
      (pickup_count >= 2)
)
SELECT category,
  AVG(delivery_time) AS avg_delivery_time
FROM delivery_differ
GROUP BY category;

**Question 3 of 3**

What is the average earnings per pickup across all routes?

Note: Some rows have missing values in the earnings column. Before calculating the final value, replace any missing earnings with the average earnings value.

In [ ]:
# Calculate average earnings per pickup to evaluate route profitability.

WITH handle_missing_value AS (
  SELECT pickup_count,
    CASE
      WHEN earnings IS NULL THEN (SELECT AVG(earnings) FROM fct_delivery_routes)
      ELSE earnings
    END AS adjusted_earnings
  FROM fct_delivery_routes
)
SELECT SUM(adjusted_earnings)/SUM(pickup_count) AS avg_per_pick_ups
FROM handle_missing_value